# LangGraph と AgentCore Memory Checkpointer（短期メモリ）

## はじめに
このノートブックでは、**AgentCoreMemorySaver** チェックポインターを使用して Amazon Bedrock AgentCore Memory 機能を LangGraph と統合する方法を紹介します。会話ターン間での**短期メモリ**の永続化に焦点を当て、自動状態チェックポイントを通じてエージェントが実行中のコンテキストを維持し、以前の計算に基づいて構築できるようにします。

## チュートリアルの詳細

| 情報               | 詳細                                                                            |
|:-------------------|:--------------------------------------------------------------------------------|
| チュートリアルタイプ | 短期会話メモリ                                                                  |
| エージェントのユースケース | 複数ステップの数学計算                                                          |
| エージェントフレームワーク | LangGraph                                                                       |
| LLM モデル          | Anthropic Claude Haiku 4.5                                                     |
| チュートリアルコンポーネント | AgentCore 短期メモリ、LangGraph Checkpointer、Math ツール                        |
| 難易度              | 初級                                                                            |

学習内容：
- 自動状態永続化のための AgentCore Memory を使用したメモリチェックポインターの作成
- AgentCore Memory バックエンドを使用した LangGraph 組み込みチェックポイントシステムの使用
- 複数のインタラクション間での会話コンテキストの維持
- 会話状態と履歴の検査と管理

### シナリオのコンテキスト

この例では、複数ステップの数学計算を実行できる「**Math エージェント**」を作成します。単純な一回限りのインタラクションとは異なり、このエージェントは AgentCore Memory のチェックポイント機能を使用して実行中のコンテキストを維持し、以前の計算に基づいて構築したり、複数のターンにわたって会話フローを記憶したりできます。

## アーキテクチャ
<div style="text-align:left">
    <img src="images/architecture.png" width="65%" />
</div>

## 前提条件

- Python 3.10以上
- 適切な権限を持つ AWS アカウント
- AgentCore Memory の適切な権限を持つ AWS IAM ロール
- Amazon Bedrock モデルへのアクセス

### 統合の仕組み

LangGraph と AgentCore Memory の統合には以下が含まれます：

1. LangGraph 状態永続化のチェックポインターバックエンドとして AgentCore Memory を使用
2. 各ステップでの会話状態の自動保存とロード
3. 複数の同時セッションとアクターのサポート

このアプローチは、手動のメモリ操作を必要とせずにシームレスな状態管理を提供し、より保守性が高くスケーラブルなエージェントアーキテクチャを実現します。

In [ ]:
# Install necessary libraries
!pip install -qr requirements.txt

In [ ]:
# Import LangGraph and LangChain components
from langchain.chat_models import init_chat_model
from langchain.tools import tool
from langgraph.prebuilt import create_react_agent

In [ ]:
# Import the AgentCoreMemorySaver that we will use as a checkpointer
import os
import logging

from langgraph_checkpoint_aws import AgentCoreMemorySaver
from bedrock_agentcore.memory import MemoryClient

region = os.getenv('AWS_REGION', 'us-west-2')
logging.getLogger("math-agent").setLevel(logging.DEBUG)

# Create or get the memory resource
memory_name = "MathLanggraphAgent"
client = MemoryClient(region_name=region)
memory = client.create_or_get_memory(name=memory_name)
memory_id = memory['id'] # Keep this memory ID for later use

### AgentCore Memory の設定

AgentCore Memory チェックポインターを設定し、LLM を初期化しましょう：

- `memory_id` はチェックポイントが保存される AgentCore Memory リソースに対応します
- `region` はリソースの AWS リージョンを指定します
- `MODEL_ID` は LangGraph エージェントを駆動する Bedrock モデルを定義します

In [ ]:
MODEL_ID = "global.anthropic.claude-haiku-4-5-20251001-v1:0"

# Initialize checkpointer for state persistence
checkpointer = AgentCoreMemorySaver(memory_id, region_name=region)

# Initialize LLM
llm = init_chat_model(MODEL_ID, model_provider="bedrock_converse", region_name=region)

### 数学ツール

エージェントが使用する数学ツールを定義しましょう。このデモでは、2つの簡単な演算を提供します：

In [ ]:
@tool
def add(a: int, b: int):
    """Add two integers and return the result"""
    return a + b


@tool
def multiply(a: int, b: int):
    """Multiply two integers and return the result"""
    return a * b


tools = [add, multiply]

### LangGraph エージェントの実装

AgentCore Memory チェックポインターを使用して LangGraph の `create_react_agent` ビルダーでエージェントを作成しましょう：

In [ ]:
graph = create_react_agent(
    model=llm,
    tools=tools,
    prompt="You are a helpful assistant",
    checkpointer=checkpointer,
)

graph

## ステップ 4: LangGraph エージェントの実行
AgentCore Memory チェックポインター統合でエージェントを実行できるようになりました。

### 設定のセットアップ
LangGraph では、config は呼び出し時に必要な属性（ユーザー ID やセッション ID など）を含む `RuntimeConfig` です。詳細は [https://langchain-ai.github.io/langgraphjs/how-tos/configuration/](https://langchain-ai.github.io/langgraphjs/how-tos/configuration/) を参照してください。

AgentCore Memory チェックポインター（`AgentCoreMemorySaver`）では、以下を指定する必要があります：
- `thread_id`: AgentCore の session_id にマップ（一意の会話スレッド）
- `actor_id`: AgentCore の actor_id にマップ（ユーザー、エージェント、またはその他の識別子）

In [ ]:
config = {
    "configurable": {
        "thread_id": "session-1", # REQUIRED: This maps to Bedrock AgentCore session_id under the hood
        "actor_id": "react-agent-1", # REQUIRED: This maps to Bedrock AgentCore actor_id under the hood
    }
}

inputs = {"messages": [{"role": "user", "content": "What is 1337 times 515321? Then add 412 and return the value to me."}]}

#### おめでとうございます！エージェントの準備ができました！

### エージェントをテストしましょう

最初の計算を実行してエージェントの動作を確認しましょう：

In [ ]:
for chunk in graph.stream(inputs, stream_mode="updates", config=config):
    print(chunk)

### エージェント状態の検査

AgentCore Memory に保存されている現在の会話状態を確認しましょう。チェックポインターは、アクターとセッションの状態を自動的に保存・取得します：

In [ ]:
for message in graph.get_state(config).values.get("messages"):
    print(f"{message.type}: {message.text()}")
    print("=========================================")

### チェックポイント履歴の表示

チェックポイント履歴を調べて、実行中にエージェントの状態がどのように変化したかを確認しましょう。チェックポイントは逆時系列順にリストされます（最新のものが最初に表示されます）。

In [ ]:
for checkpoint in graph.get_state_history(config):
    print(
        f"(Checkpoint ID: {checkpoint.config['configurable']['checkpoint_id']}) # of messages in state: {len(checkpoint.values.get('messages'))}"
    )

### メモリ永続化のテスト

会話を続けてチェックポインターの威力をテストしましょう。エージェントは以前の計算を記憶しているはずです：

In [ ]:
inputs = {"messages": [{"role": "user", "content": "What were the first calculations I asked you to do?"}]}

for chunk in graph.stream(inputs, stream_mode="updates", config=config):
    print(chunk)

### 新しいセッションの開始

新しい会話スレッドを作成してセッションの分離を実演しましょう。この新しいセッションでは、エージェントは以前の計算を記憶していません：

In [ ]:
config = {
    "configurable": {
        "thread_id": "session-2", # New session ID
        "actor_id": "react-agent-1", # Same Actor ID
    }
}

inputs = {"messages": [{"role": "user", "content": "What values did I ask you to multiply and add?"}]}
for chunk in graph.stream(inputs, stream_mode="updates", config=config):
    print(chunk)

## まとめ

このノートブックでは、以下を実演しました：

1. チェックポイント用の AgentCore Memory リソースの作成方法
2. 自動状態永続化を持つ LangGraph エージェントの構築
3. 複数ステップ計算用の数学ツールの実装
4. チェックポインターバックエンドとしての AgentCoreMemorySaver の使用
5. メモリ永続化とセッション分離のテスト

この統合は、LangGraph の構造化されたワークフローと AgentCore Memory の堅牢なチェックポイント機能を組み合わせることで、複数のインタラクション間でコンテキストを維持できるステートフルで永続的な AI エージェントを作成する力を示しています。

ここで実演したアプローチは、マルチエージェントシステム、長時間実行ワークフロー、会話コンテキストに基づく特殊な状態管理など、より複雑なユースケースに拡張できます。

### クリーンアップ
このノートブックで使用したリソースをクリーンアップするためにメモリを削除しましょう。

In [ ]:
#client.delete_memory_and_wait(memory_id = memory_id, max_wait = 300, poll_interval =10)